In [5]:
import torch
import torchvision
import torchvision.transforms as transforms

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((224,224))])
batch_size = 16

In [11]:
direc = "../imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/"

In [12]:
torchvision.datasets.ImageNet(root=direc, split = 'train')

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ../imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/.

In [ ]:
class ImageNet_dataset(Dataset):
    """ImageNet dataset."""

    def __init__(self,root_dir, transform=None, phase = ""):
        """
        Arguments:
            root_dir (string): Directory with all the downloaded dataset.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.root_dir = root_dir
        self.transform = transform
        self.phase = phase
        
        self.X = {}
        self.Y = {}
        
        X_train, Y_train, X_test, Y_test = load_CIFAR10(root_dir)
        
        self.X["train"] = X_train
        self.X["test"] = X_test
        
        self.Y["train"] = Y_train
        self.Y["test"] = Y_test
        
    def __len__(self):
            return len(self.X[self.phase])
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.X[self.phase][idx]
        image = image.reshape(3,32,32)
        image = image.transpose(1,2,0)

        y = self.Y[self.phase][idx]
        y = np.array(y)


        if self.transform:
            image = self.transform(image)

        return image,y,idx

In [16]:
"../imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/"

../imagenet-object-localization-challenge.zip

../imagenet-object-localization-challenge:
ILSVRC/                    LOC_synset_mapping.txt  LOC_val_solution.csv
LOC_sample_submission.csv  LOC_train_solution.csv


In [17]:
torchvision.datasets.ImageNet(root= , split = 'val')

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ../imagenet-object-localization-challenge.zip.

In [3]:
trainset = torchvision.datasets.GTSRB("/..", split = 'train', download= True)

AttributeError: module 'torchvision.datasets' has no attribute 'GTSRB'

In [4]:
import torchvision
print(torchvision.__version__)

0.11.2
